In [91]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [94]:
"""
60191672 윤태균 
기능 설명: a->csv 파일을 dataframe형태로 load하고 shape와 첫 20행을 출력
           b->1~5열에서 0값을 NaN으로 치환하고 첫 20행을 출력/NaN으로 치환된 데이터 
              셀의 개수 출력
           c->column0 을 0회는 A로 1~3회는 B로 3회 이상은 C로 그룹을 나누어 column0
              에 표기한다. 그 후 ColumnTransfer를 이용하여 numpy array를 출력하고,
              dataframe으로 변환하여 출력하고, 마지막으로 sparse matrix를 출력한다.
           d->결측치 NaN으로 표시된 행을 모두 삭체하고 삭체 전/후 행과 열의 개수출력
           e->각 결측치들을 각 column별 평균으로 바꾸고 결측치들이 있던 행을 출력하고
              행의 개수도 출력한다.
           f->우선 당뇨병의 유무를 positive와 negative로 나누어 groupby를 한다. 그 후 
              각 그룹에서 50개 씩 sampling을 복원없이 추출하여 2개를 concat하여 100개의 
              표본을 가진 2개의 파일을 만든다.
"""


from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from scipy import sparse
from pandas import DataFrame
import pandas as pd
import numpy as np

dataset = pd.read_csv('pima-indians-diabetes.csv', header = None)

print("(a)\n")
print(dataset.shape)
print(dataset.head(20))

print("\n(b)\n")
dataset[[1,2,3,4,5]] = dataset[[1,2,3,4,5]].replace(0,np.NaN)
print(dataset.head(20))
print("NaN으로 치환된 데이터 셀의 개수는",dataset.isnull().sum().sum(),"개 입니다.")

print("\n(c)\n")
dataset0 = dataset
#'A'그룹은 0번 임심  'B'그룹은 1,2번 임신 'C'그룹은 3번 이상 임신 (0column)
dataset0.loc[dataset0[0]==0,'group'] = 'A'
dataset0.loc[(dataset0[0] >=1)&(dataset0[0]<3),'group'] = 'B'
dataset0.loc[dataset0[0] >=3,'group'] = 'C'
dataset0[0] = dataset0['group']
dataset0 = dataset0.drop(['group'],axis =1)
print(dataset0[374:394]) 
print(type(dataset0))

ct = ColumnTransformer([('one_hot_encoder',OneHotEncoder(),[0])],remainder='passthrough')
dataset1 = ct.fit_transform(dataset0)
print(dataset1[374:394]) #numpy array
print(type(dataset1))
dataset2 = pd.DataFrame(dataset1)
print(dataset2[374:394])
print(type(dataset2)) #dataframe
dataset3 = sparse.csr_matrix(dataset1[374:394])
print(dataset3)
print(type(dataset3)) #Sparse matrix

print("\n(d)\n")
dataset4 = dataset0.dropna(inplace = False)
print(dataset0.shape)
print(dataset4.shape)

print("\n(e)\n")
dataset5 = dataset0.fillna(dataset0.mean(),inplace = False)
null_data = dataset0[dataset0.isnull().any(axis=1)]
print(null_data)
print("치환이 발생한 행의 개수",null_data.shape[0])

print("\n(f)\n")
df = pd.read_csv('pima-indians-diabetes.csv', header = None)
df_0 = df.groupby(8).get_group(0)
df_1 = df.groupby(8).get_group(1)

df_train_x = df_0.sample(n =50, replace = False)
df_train_y = df_1.sample(n =50, replace = False)
df_train = pd.concat([df_train_x,df_train_y])
df_train.to_csv('df_train.csv',sep=',',na_rep='NaN',header=True)

df_test_x = df_0.sample(n =50, replace = False)
df_test_y = df_1.sample(n =50, replace = False)
df_test = pd.concat([df_test_x,df_test_y])
df_test.to_csv('df_test.csv',sep=',',na_rep='NaN',header=True)



'\n60191672 윤태균 \n기능 설명: a->csv 파일을 dataframe형태로 load하고 shape와 첫 20행을 출력\n           b->1~5열에서 0값을 NaN으로 치환하고 첫 20행을 출력/NaN으로 치환된 데이터 \n              셀의 개수 출력\n           c->column0 을 0회는 A로 1~3회는 B로 3회 이상은 C로 그룹을 나누어 column0\n              에 표기한다. 그 후 ColumnTransfer를 이용하여 numpy array를 출력하고,\n              dataframe으로 변환하여 출력하고, 마지막으로 sparse matrix를 출력한다.\n           d->결측치 NaN으로 표시된 행을 모두 삭체하고 삭체 전/후 행과 열의 개수출력\n           e->각 결측치들을 각 column별 평균으로 바꾸고 결측치들이 있던 행을 출력하고\n              행의 개수도 출력한다.\n           f->우선 당뇨병의 유무를 positive와 negative로 나누어 groupby를 한다. 그 후 \n              각 그룹에서 50개 씩 sampling을 복원없이 추출하여 2개를 concat하여 100개의 \n              표본을 가진 2개의 파일을 만든다.\n'

(a)

(768, 9)
     0    1   2   3    4     5      6   7  8
0    6  148  72  35    0  33.6  0.627  50  1
1    1   85  66  29    0  26.6  0.351  31  0
2    8  183  64   0    0  23.3  0.672  32  1
3    1   89  66  23   94  28.1  0.167  21  0
4    0  137  40  35  168  43.1  2.288  33  1
5    5  116  74   0    0  25.6  0.201  30  0
6    3   78  50  32   88  31.0  0.248  26  1
7   10  115   0   0    0  35.3  0.134  29  0
8    2  197  70  45  543  30.5  0.158  53  1
9    8  125  96   0    0   0.0  0.232  54  1
10   4  110  92   0    0  37.6  0.191  30  0
11  10  168  74   0    0  38.0  0.537  34  1
12  10  139  80   0    0  27.1  1.441  57  0
13   1  189  60  23  846  30.1  0.398  59  1
14   5  166  72  19  175  25.8  0.587  51  1
15   7  100   0   0    0  30.0  0.484  32  1
16   0  118  84  47  230  45.8  0.551  31  1
17   7  107  74   0    0  29.6  0.254  31  1
18   1  103  30  38   83  43.3  0.183  33  0
19   1  115  70  30   96  34.6  0.529  32  1

(b)

     0      1     2     3      4   

In [95]:

print(df_train.shape)
print(df_test.shape)

(100, 9)
(100, 9)
